In [1]:
import numpy as np
import os 
import arviz as az
import pandas as pd
import math
import xarray as xr
os.chdir('..')
import waic as wc
import model_data as md

In [4]:
os.chdir('../data')
fnames = ['sockeye_data.csv']
df = md.load_data(fnames)

C:\Users\brett\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
os.chdir('../Pystan/cmdstan_output')

In [6]:
cued_basic_fit = az.from_netcdf('cue_basic_fit.nc')
classic_basic_fit = az.from_netcdf('Classic_basic_classic_fit.nc')

cued_pscale_fit = az.from_netcdf('cue_pscale_fit.nc')
classic_pscale_fit = az.from_netcdf('Classic_pscale_fit.nc')

cued_pindep_fit = az.from_netcdf('cue_pindep_fit.nc')
classic_pindep_fit = az.from_netcdf('Classic_pindep_fit.nc')

cued_basicstar_fit = az.from_netcdf('cue_basicstar_fit.nc')
classic_basicstar_fit = az.from_netcdf('Classic_basicstar_fit.nc')

cued_pscalestar_fit = az.from_netcdf('cue_pscale_fit.nc')
classic_pscalestar_fit = az.from_netcdf('Classic_pscalestar_fit.nc')

cued_pindepstar_fit = az.from_netcdf('cue_pindepstar_fit.nc')
classic_pindepstar_fit = az.from_netcdf('Classic_pindepstar_fit.nc')

In [7]:
task_list = df.groupby(['MSN'])['Subject'].unique()

In [8]:
task_list

MSN
RGT_ClassicA     [225, 226, 227, 228, 509, 510, 511, 512, 625, ...
RGT_ClassicB     [229, 230, 231, 232, 513, 514, 515, 516, 613, ...
RIGT_ClassicA                                      [901, 902, 916]
RIGT_ClassicB                                                [915]
rGT_A-cue        [173, 174, 177, 178, 179, 180, 329, 330, 331, ...
rGT_B-cue        [175, 181, 182, 183, 184, 325, 326, 327, 328, ...
rGT_ClassicA     [201, 202, 217, 218, 219, 220, 233, 234, 235, ...
rGT_ClassicB     [207, 208, 221, 222, 223, 224, 237, 238, 239, ...
Name: Subject, dtype: object

In [9]:
classic_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])
cued_subs = np.concatenate((task_list['rGT_A-cue'], task_list['rGT_B-cue']))

In [10]:
len(classic_subs)

78

In [11]:
from timeit import default_timer as timer

start = timer()

cued_basic_waic = wc.waic_fit(df, 5, cued_subs, cued_basic_fit, "basic")
end = timer()
print(end - start) # Time in seconds


##changed waic.py to be 3200 instead of 4000 parameter estimates 

C:\Users\brett\sparklyRGT\Pystan\model_data.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Option'][i] = 0
C:\Users\brett\sparklyRGT\Pystan\model_data.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sub'][i] = sub_dict[df['Subject'][i]]


KeyboardInterrupt: 

In [ ]:
cued_basic_waic = wc.waic_fit(df, 5, cued_subs, cued_basic_fit, "basic")
classic_basic_waic = wc.waic_fit(df, 5, classic_subs, classic_basic_fit, "basic")

basic_list = [cued_basic_waic.waic, classic_basic_waic.waic]
basic_list_SE = [cued_basic_waic.waic_se, classic_basic_waic.waic_se]

In [ ]:
cued_basicstar_waic = wc.waic_fit(df, 5, cued_subs, cued_basicstar_fit, "basicstar")
classic_basic_waic = wc.waic_fit(df, 5, classic_subs, classic_basicstar_fit, "basicstar")

basicstar_list = [cued_basicstar_waic.waic, classic_basicstar_waic.waic]
basicstar_list_SE = [cued_basicstar_waic.waic_se, classic_basicstar_waic.waic_se]

In [ ]:
cued_pscale_waic = wc.waic_fit(df, 5, cued_subs, cued_pscale_fit, "pscale")
classic_pscale_waic = wc.waic_fit(df, 5, classic_subs, classic_pscale_fit, "pscale")

pscale_list = [cued_pscale_waic.waic, classic_pscale_waic.waic]
pscale_list_SE = [cued_pscale_waic.waic_se, classic_pscale_waic.waic_se]

In [ ]:
cued_pscalestar_waic = wc.waic_fit(df, 5, cued_subs, cued_pscalestar_fit, "pscalestar")
classic_pscalestar_waic = wc.waic_fit(df, 5, classic_subs, classic_pscalestar_fit, "pscalestar")

pscalestar_list = [cued_pscalestar_waic.waic, classic_pscalestar_waic.waic]
pscalestar_list_SE = [cued_pscalestar_waic.waic_se, classic_pscalestar_waic.waic_se]

In [ ]:
cued_pindep_waic = wc.waic_fit(df, 5, cued_subs, cued_pindep_fit, "pindep")
classic_pindep_waic = wc.waic_fit(df, 5, classic_subs, classic_pindep_fit, "pindep")

pindep_list = [cued_pindep_waic.waic, classic_pindep_waic.waic]
pindep_list_SE = [cued_pindep_waic.waic_se, classic_pindep_waic.waic_se]

In [ ]:
cued_pindepstar_waic = wc.waic_fit(df, 5, cued_subs, cued_pindepstar_fit, "pindepstar")
classic_pindepstar_waic = wc.waic_fit(df, 5, classic_subs, classic_pindepstar_fit, "pindepstar")

pindep_list = [cued_pindepstar_waic.waic, classic_pindepstar_waic.waic]
pindep_list_SE = [cued_pindepstar_waic.waic_se, classic_pindepstar_waic.waic_se]

In [ ]:
waic_values = pd.DataFrame(
    {'basic_waic': basic_list,
     'basic_waic_SE': basic_list_SE, 
     'basicstar_waic': basicstar_list,
     'basicstar_waic_SE': basicstar_list_SE,
     'pscale_waic': pscale_list,
     'pscale_waic_SE': pscale_list_SE,
     'pscalestar_waic': pscalestar_list,
     'pscalestar_SE': pscalestar_list_SE,
     'pindep_waic': pindep_list,
     'pindep_waic_SE': pindep_list_SE,
     'pindepstar_waic': pindepstar_list,
     'pindepstar_waic_SE': pindepstar_list_SE},
     index = ["cued", "classic"])

waic_values.to_excel("langdon_waic_values.xlsx")